# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [43]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [20]:
# constants

MODEL_GPT = 'gpt-3.5-turbo'
MODEL_LLAMA = 'llama3.2'

In [8]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")


In [9]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

system_prompt = """
You're a senior developer bot that answers technical question. Your answers need to explain the theory, provide examples, and a list of link to the documentation. Using markdown.
"""


In [31]:
def get_user_prompt(question):
    user_prompt = "You're receiving this question to answer:\n";
    user_prompt += question
    
    return user_prompt


In [19]:
openai = OpenAI()

In [44]:
# Get gpt-4o-mini to answer, with streaming

def answer_question(question):
    print(question)
    stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = [
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": get_user_prompt(question) },
        ],
        stream = True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [45]:
answer_question(question)


Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}



This code snippet is using a combination of Python's `yield from`, generator expressions, and dictionary comprehension to yield the author values from a list of dictionaries (`books`) where the author key exists and is not None or False.

Let's break it down step by step:

1. `{book.get("author") for book in books if book.get("author")}` (dictionary comprehension): Here, we iterate over each `book` in the `books` list. For each `book`, we retrieve the value corresponding to the key "author" using `book.get("author")`. If the value is not None or False (this condition is checked by the `if book.get("author")` part), then that value is included in the set that is created. Due to it being a set comprehension, this will result in a set of unique author values where the author is not None or False.

2. `yield from`: The `yield from` statement is used to delegate the yield value to another generator. By using `yield from`, the values yielded by the set comprehension are being yielded to the calling function or generator where this code resides.

So, in essence, this code generates a sequence of unique author values from the `books` list after checking that the author is not None or False.

Here's an example to illustrate this:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author B"},
    {"title": "Book 4"},
    {"title": "Book 5", "author": "Author A"}
]

def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

unique_authors = get_unique_authors(books)
for author in unique_authors:
    print(author)


Output:

Author A
Author B


For more information, you can refer to the Python documentation on:
- `yield from` statement: [Python Documentation - The yield from expression](https://docs.python.org/3/reference/expressions.html#yield-from)
- Dictionary Comprehension: [Python Documentation - Dict Comprehensions](https://docs.python.org/3/tutorial/datastructures.html#dictionaries)
- `set` type in Python: [Python Documentation - Set Types](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset)

In [47]:
import ollama

In [49]:
# Get Llama 3.2 to answer


# Get gpt-4o-mini to answer, with streaming

def answer_question_ollama(question):
    response = ollama.chat(
        model = MODEL_LLAMA,
        messages = [
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": get_user_prompt(question) },
        ]
    )
    display(Markdown(response['message']['content']))

In [50]:
answer_question_ollama(question)

**Code Explanation**
=====================

The given code snippet uses a combination of Python features, including generators, dictionary methods, and list comprehensions. Let's break it down:

```python
from typing import Generator, Dict

def get_books(books: List[Dict]) -> Generator[str, None, None]:
    yield from {book.get("author") for book in books if book.get("author")}
```

In this context, `get_books` is a function that takes a list of dictionaries (`books`) as input and yields a generator of authors.

**Here's what it does:**

1. The dictionary comprehension `{book.get("author") for book in books if book.get("author")}` iterates over each dictionary (`book`) in the `books` list.
2. For each `book`, it attempts to retrieve the value associated with the key `"author"`. If this key exists, its corresponding value is yielded as a generator element.
3. The `yield from` statement takes the generator returned by the comprehension and yields each of its elements directly.

**Why?**
--------

The use of `yield from` allows us to delegate the iteration over the dictionary comprehension to another generator or iterator. This has several benefits:

*   **Lazy evaluation**: The values are only computed when an iteration is requested, which can be more memory-efficient for large datasets.
*   **Flexibility**: We can easily modify or extend the comprehension without affecting the overall generator.

**Example Use Case**
--------------------

Suppose we have a list of book dictionaries:
```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "publisher": "Publisher B"},
    {"title": "Book 3", "author": "Author C"}
]
```
Using the `get_books` function, we can retrieve a generator of authors:
```python
authors = get_books(books)
for author in authors:
    print(author)  # Output: Author A, Author C
```

**Documentation Links**
---------------------

*   [Python Dictionary Methods](https://docs.python.org/3/library/stdtypes.html#dict-methods)
*   [List Comprehensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions)
*   [Generators in Python](https://docs.python.org/3/reference/datamodel.html#generator-objects)